In [44]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import os
from bs4 import BeautifulSoup
from tqdm import tqdm

In [88]:
chrome_path = Service(r'./chromedriver/chromedriver')
driver = webdriver.Chrome(service=chrome_path)

In [89]:
ids = pd.read_csv('data/Link_ID.csv',names=['id'])
ids=list(ids['id'])

In [47]:
electoral_XPATH = '//*[@id="headingOne"]/h2/a'

download_XPATH = '//*[@id="gridCandidates"]/div/div[4]/div/div/div[3]/div[3]/div/div/div'
download_XPATH2 = '//*[@id="gridCandidates"]/div/div[4]/div/div/div[3]/div[3]/div/div'
name_PATH = r'//*[@id="PersonTitle"]/h2'

* I saw that we can start from 4213

#### not downloading

In [90]:
def get_electoral(ID):
    web = 'https://lop.parl.ca/sites/ParlInfo/default/en_CA/People/Profile?personId='+str(ID)
    driver.get(web)
    time.sleep(1)
    
    try:
        name = driver.find_element(By.XPATH, name_PATH).text
    except: 
        name = '-'
    
    time.sleep(1)
    
    try:
        
        time.sleep(1)
        
        electoral = driver.find_element(By.XPATH, electoral_XPATH)
        electoral.click()
        
        time.sleep(1)

        try:
            table = driver.find_element(By.XPATH , '//*[@id="gridCandidates"]/div/div[6]/div/div/div/div/table/tbody')

        except:
            time.sleep(2)
            table = driver.find_element(By.XPATH , '//*[@id="gridCandidates"]/div/div[6]/div/div/div/div/table/tbody')

        column_names = ['Parliament','Election Date','Constituency','Party','Result','Votes']
        rows=table.text.split('\n') 

        data = []
        for row in rows:
            row_instances = row.split(' ')
            Votes = row_instances.pop()
            Result = row_instances.pop()
            Parliament = row_instances.pop(0)
            Election_Date = row_instances.pop(0)
            Constituency = row_instances.pop(0)
            Party = ' '.join(row_instances)
            data.append([Parliament, Election_Date, Constituency, Party, Result, Votes])

        temp_df = pd.DataFrame(data=data,columns=column_names)
        temp_df['name']=str(name)
        temp_df['id']=str(ID)
    
    except: 

        print(str(ID) + ' do not have electoral data')      
    
    return temp_df

In [91]:
def get_electorals(ids):
    df_electoral = pd.DataFrame()
    for ID in tqdm(ids):
        try:
            df_electoral = pd.concat([df_electoral,get_electoral(ID)])
        except Exception as e:
            print('error for ID: ' + str(ID) + ' is ' + str(e))
    return df_electoral

In [ ]:
df_electoral=get_electorals(ids[4700:4800])

In [93]:
df_electoral.to_csv('electoral_data.csv')

#### with download button

In [ ]:
def get_electoral(ID):

    web = 'https://lop.parl.ca/sites/ParlInfo/default/en_CA/People/Profile?personId='+str(ID)
    driver.get(web)

    time.sleep(1)    

    name = driver.find_element(By.XPATH, name_PATH).text

    time.sleep(1)

    electoral = driver.find_element(By.XPATH, electoral_XPATH)
    electoral.click()

    time.sleep(1)

    download = driver.find_element(By.XPATH, download_XPATH2)
    download.click()

    time.sleep(5)

    # if there is no error until here, I have to check it
    temp_df = pd.read_excel('/Users/batuhan_mac/Downloads/Parliamentarian Profile - Electoral History.xlsx')
    temp_df['id']=ID
    temp_df['name']=name
    os.remove("/Users/batuhan_mac/Downloads/Parliamentarian Profile - Electoral History.xlsx")

    return temp_df